In [1]:
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.53.0
!uv pip install --no-deps trl==0.22.2

Using Python 3.11.13 environment at: /usr
Resolved 30 packages in 18ms                                         
Uninstalled 2 packages in 85ms
Installed 2 packages in 73ms                                
 - tokenizers==0.22.1
 + tokenizers==0.21.4
 - transformers==4.56.2
 + transformers==4.53.0
Using Python 3.11.13 environment at: /usr
Resolved 1 package in 1ms                                            
Uninstalled 1 package in 3ms
Installed 1 package in 8ms                                  
 - trl==0.23.0
 + trl==0.22.2


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-1.7B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.9, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-09-26 19:13:27.040060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758914007.438122     167 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758914007.551566     167 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 09-26 19:13:55 [__init__.py:244] Automatically detected platform cuda.
ERROR 09-26 19:13:57 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 09-26 19:14:05 [vllm_utils.py:689] Unsloth: Patching vLLM v1 graph capture
INFO 09-26 19:14:05 [vllm_utils.py:717] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.9.9: Fast Qwen3 patching. Transformers: 4.53.0. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-1.7B-Base with actual GPU utilization = 89.15%
Unslo

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.9.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [3]:
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np

In [4]:
dataset = load_dataset("openlifescienceai/medmcqa", split="train")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/85.9M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/936k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [5]:
# Chat Template for GRPO
explanation_start = "<explanation>"
explanation_end   = "</explanation>"
solution_start    = "<answer>"
solution_end      = "</answer>"

system_prompt = f"""The User asks a multiple choice question, and the Assistant answers the question from the given options. Think about the problem, pick the option that answers the question and provide your explanation for choosing that option.

Firstly, provide your answer between {solution_start}{solution_end}.
Then place your explanation between {explanation_start} and {explanation_end}.

Example format:
{solution_start}0{solution_end}
{explanation_start}Your reasoning here{explanation_end}"""

In [6]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{solution_start}'")
tokenizer.chat_template = chat_template

In [7]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{solution_start}2{solution_end}{explanation_start}I think's it is 2.{explanation_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

"The User asks a multiple choice question, and the Assistant answers the question from the given options. Think about the problem, pick the option that answers the question and provide your explanation for choosing that option.\n\nFirstly, provide your answer between <answer></answer>.\nThen place your explanation between <explanation> and </explanation>.\n\nExample format:\n<answer>0</answer>\n<explanation>Your reasoning here</explanation><|endoftext|>What is 1+1?<answer>2</answer><explanation>I think's it is 2.</explanation><|endoftext|>What is 2+2?<answer>"

In [8]:
dataset = dataset.to_pandas()[
    ["question", "opa", "opb","opc","opd","cop","exp"]
]

In [9]:
def format_dataset(x):
    correct_answer_index = x["cop"]
    options = [x["opa"], x["opb"], x["opc"], x["opd"]]
    correct_answer = options[correct_answer_index]

    # The user prompt is the question + options
    #question = x["question"]
    question = f"""{x["question"]}
            Options:
            0. {x["opa"]}
            1. {x["opb"]}
            2. {x["opc"]}
            3. {x["opd"]}"""

    # Get the Explanations. Some of the questions also have no explanation to them
    explanation = x["exp"]
    if explanation is None or str(explanation).lower() in ["none", "nan", "null", ""]:
        explanation = ""
    explanation = str(explanation).strip()


    # Add our custom formatting
    final_prompt = \
        solution_start + str(correct_answer_index)  + solution_end + \
        explanation_start + explanation + explanation_end 

    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : question},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

In [ ]:
dataset["Messages"][2]

In [10]:
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

'The User asks a multiple choice question, and the Assistant answers the question from the given options. Think about the problem, pick the option that answers the question and provide your explanation for choosing that option.\n\nFirstly, provide your answer between <answer></answer>.\nThen place your explanation between <explanation> and </explanation>.\n\nExample format:\n<answer>0</answer>\n<explanation>Your reasoning here</explanation><|endoftext|>Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\n            Options:\n            0. Hyperplasia\n            1. Hyperophy\n            2. Atrophy\n            3. Dyplasia<answer>2</answer><explanation>Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with

In [11]:
dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['question', 'opa', 'opb', 'opc', 'opd', 'cop', 'exp', 'Messages', 'text'],
    num_rows: 182822
})

In [12]:
dataset['text'][3]

"The User asks a multiple choice question, and the Assistant answers the question from the given options. Think about the problem, pick the option that answers the question and provide your explanation for choosing that option.\n\nFirstly, provide your answer between <answer></answer>.\nThen place your explanation between <explanation> and </explanation>.\n\nExample format:\n<answer>0</answer>\n<explanation>Your reasoning here</explanation><|endoftext|>Following endaerectomy on the right common carotid, a patient is found to be blind in the right eye. It is appears that a small thrombus embolized during surgery and lodged in the aery supplying the optic nerve. Which aery would be blocked?\n            Options:\n            0. Central aery of the retina\n            1. Infraorbital aery\n            2. Lacrimal aery\n            3. Nasociliary aretry<answer>0</answer><explanation>The central aery of the retina is a branch of the ophthalmic aery. It is the sole blood supply to the retina

In [13]:
# To get the model used to the prompt style, we will only train it on 1000 questions
subset_size = 1000  # pick a few thousand
dataset_small = dataset.shuffle(seed=42).select(range(subset_size))

In [ ]:
len(dataset_small)

In [14]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_small,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 2 | Total steps = 2,000
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 34,865,152 of 1,755,440,128 (1.99% trained)


Step,Training Loss
5,2.104800
10,1.748100
15,1.109800
20,0.818100
25,1.232100
30,1.232600
35,1.436100
40,0.915800
45,0.893500
50,1.255400


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=2000, training_loss=0.9370754218101501, metrics={'train_runtime': 808.2481, 'train_samples_per_second': 2.474, 'train_steps_per_second': 2.474, 'total_flos': 4679451648000000.0, 'train_loss': 0.9370754218101501, 'epoch': 2.0})

In [16]:
text = tokenizer.apply_chat_template(
    dataset[178]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0.7,
    max_new_tokens = 512,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

The User asks a multiple choice question, and the Assistant answers the question from the given options. Think about the problem, pick the option that answers the question and provide your explanation for choosing that option.

Firstly, provide your answer between <answer></answer>.
Then place your explanation between <explanation> and </explanation>.

Example format:
<answer>0</answer>
<explanation>Your reasoning here</explanation><|endoftext|>Symptomatic treatment is not required in withdrawal of:
            Options:
            0. Cannabis
            1. Morphine
            2. Alcohol
            3. Cocaine<answer>1</answer><explanation>Ans. is 'b' i.e., Morphine</explanation><|endoftext|>


In [17]:
dataset = [
    {
        "Messages": [
            {
                "role": "system",
                "content": """The User asks a multiple choice question, and the Assistant answers the question from the given options.
The Assistant first thinks about the reasoning process in the mind and then provides the User with the final answer.
The output format of final answer and explanation process are enclosed within <answer> </answer> and <explanation> </explanation> tags."""
            },
            {
                "role": "user",
                "content": """Question: Which of the following is the first line treatment for acute myocardial infarction?
Options:
0. Aspirin
1. Morphine
2. Oxygen
3. Nitroglycerin"""
            }
        ]
    },
    {
        "Messages": [
            {
                "role": "system",
                "content": """The User asks a multiple choice question, and the Assistant answers the question from the given options.
The Assistant first thinks about the reasoning process in the mind and then provides the User with the final answer.
The output format of final answer and explanation process are enclosed within <answer> </answer> and <explanation> </explanation> tags."""
            },
            {
                "role": "user",
                "content": """Question: The most common cause of community-acquired pneumonia in adults is:
Options:
0. Streptococcus pneumoniae
1. Haemophilus influenzae
2. Mycoplasma pneumoniae
3. Staphylococcus aureus"""
            }
        ]
    },
    {
        "Messages": [
            {
                "role": "system",
                "content": """The User asks a multiple choice question, and the Assistant answers the question from the given options.
The Assistant first thinks about the reasoning process in the mind and then provides the User with the final answer.
The output format of final answer and explanation process are enclosed within <answer> </answer> and <explanation> </explanation> tags."""
            },
            {
                "role": "user",
                "content": """Question: Normal blood pressure range for adults is:
Options:
0. <120/80 mmHg
1. <130/85 mmHg
2. <140/90 mmHg
3. <150/95 mmHg"""
            }
        ]
    },
    {
        "Messages": [
            {
                "role": "system",
                "content": """The User asks a multiple choice question, and the Assistant answers the question from the given options.
The Assistant first thinks about the reasoning process in the mind and then provides the User with the final answer.
The output format of final answer and explanation process are enclosed within <answer> </answer> and <explanation> </explanation> tags."""
            },
            {
                "role": "user",
                "content": """Question: Which hormone is primarily responsible for milk ejection reflex?
Options:
0. Prolactin
1. Oxytocin
2. Estrogen
3. Growth hormone"""
            }
        ]
    },
    {
        "Messages": [
            {
                "role": "system",
                "content": """The User asks a multiple choice question, and the Assistant answers the question from the given options.
The Assistant first thinks about the reasoning process in the mind and then provides the User with the final answer.
The output format of final answer and explanation process are enclosed within <answer> </answer> and <explanation> </explanation> tags."""
            },
            {
                "role": "user",
                "content": """Question: The gold standard test for diagnosing pulmonary embolism is:
Options:
0. CT pulmonary angiography
1. Chest X-ray
2. D-dimer
3. ECG"""
            }
        ]
    }
]

In [18]:
text = tokenizer.apply_chat_template(
    dataset[4]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 1,
    max_new_tokens = 256,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

The User asks a multiple choice question, and the Assistant answers the question from the given options.
The Assistant first thinks about the reasoning process in the mind and then provides the User with the final answer.
The output format of final answer and explanation process are enclosed within <answer> </answer> and <explanation> </explanation> tags.<|endoftext|>Question: The gold standard test for diagnosing pulmonary embolism is:
Options:
0. CT pulmonary angiography
1. Chest X-ray
2. D-dimer
3. ECG<answer>0</answer><explanation>Ans. is 'a' i.e., CT pulmonary angiography</explanation><|endoftext|>


In [19]:
save_dir = "./pre_finetuned_model_1.7B"

trainer.save_model(save_dir)   # saves model + adapter weights (if any)
tokenizer.save_pretrained(save_dir)  # saves tokenizer files too

('./pre_finetuned_model_1.7B/tokenizer_config.json',
 './pre_finetuned_model_1.7B/special_tokens_map.json',
 './pre_finetuned_model_1.7B/chat_template.jinja',
 './pre_finetuned_model_1.7B/vocab.json',
 './pre_finetuned_model_1.7B/merges.txt',
 './pre_finetuned_model_1.7B/added_tokens.json',
 './pre_finetuned_model_1.7B/tokenizer.json')

In [30]:
save_dir = "/kaggle/working/"

# Fine Tuned Model for Prompt Response Style
!tar -czvf pre_finetuned_model_1.7B.tar.gz pre_finetuned_model_1.7B

pre_finetuned_model_1.7B/
pre_finetuned_model_1.7B/special_tokens_map.json
pre_finetuned_model_1.7B/tokenizer.json
pre_finetuned_model_1.7B/added_tokens.json
pre_finetuned_model_1.7B/adapter_config.json
pre_finetuned_model_1.7B/merges.txt
pre_finetuned_model_1.7B/README.md
pre_finetuned_model_1.7B/chat_template.jinja
pre_finetuned_model_1.7B/adapter_model.safetensors
pre_finetuned_model_1.7B/tokenizer_config.json
pre_finetuned_model_1.7B/training_args.bin
pre_finetuned_model_1.7B/vocab.json


In [31]:
del dataset
torch.cuda.empty_cache()
import gc
gc.collect()

694

In [32]:
def prepare_dataset_medmcqa_grpo(split="train"):
    data = load_dataset("openlifescienceai/medmcqa", split=split)
    formatted_data = []
    
    for example in data:
        question = f"""{example["question"]}
            Options:
            0. {example["opa"]}
            1. {example["opb"]}
            2. {example["opc"]}
            3. {example["opd"]}"""

        correct_answer_index = example["cop"]
        options = [example["opa"], example["opb"], example["opc"], example["opd"]]
        correct_answer = options[correct_answer_index]
        
        formatted_example = {
            "prompt": question,
            "answer": str(correct_answer_index),  
        }
        formatted_data.append(formatted_example)

    return formatted_data

dataset_grpo = prepare_dataset_medmcqa_grpo(split="train")

In [33]:
dataset_grpo[2]

{'prompt': 'All of the following are surgical options for morbid obesity except -\n            Options:\n            0. Adjustable gastric banding\n            1. Biliopancreatic diversion\n            2. Duodenal Switch\n            3. Roux en Y Duodenal By pass',
 'answer': '3'}

In [34]:
dataset_grpo[2]["prompt"]

'All of the following are surgical options for morbid obesity except -\n            Options:\n            0. Adjustable gastric banding\n            1. Biliopancreatic diversion\n            2. Duodenal Switch\n            3. Roux en Y Duodenal By pass'

In [35]:
dataset_grpo[2]["answer"]

'3'

In [36]:
from datasets import Dataset

dataset_grpo = Dataset.from_list(dataset_grpo)

In [37]:
dataset_grpo = dataset_grpo.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["prompt"]},
    ],
    "answer": x["answer"],
})
dataset_grpo[0]

Map:   0%|          | 0/182822 [00:00<?, ? examples/s]

{'prompt': [{'content': 'The User asks a multiple choice question, and the Assistant answers the question from the given options. Think about the problem, pick the option that answers the question and provide your explanation for choosing that option.\n\nFirstly, provide your answer between <answer></answer>.\nThen place your explanation between <explanation> and </explanation>.\n\nExample format:\n<answer>0</answer>\n<explanation>Your reasoning here</explanation>',
   'role': 'system'},
  {'content': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\n            Options:\n            0. Hyperplasia\n            1. Hyperophy\n            2. Atrophy\n            3. Dyplasia',
   'role': 'user'}],
 'answer': '2'}

In [38]:
# Save Hugging Face dataset
dataset_grpo.save_to_disk("medmcqa_grpo_formatted")

# Save JSON
dataset_grpo.to_json("medmcqa_grpo_formatted.json", orient="records", lines=True)

Saving the dataset (0/1 shards):   0%|          | 0/182822 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/183 [00:00<?, ?ba/s]

141542127

In [39]:
data = dataset_grpo.train_test_split(0.02,seed=21)
train = data['train'].shuffle(seed=42).select(range(40000))
validation = data['test'].shuffle(seed=42).select(range(3000))

In [40]:
import re

In [41]:
def reward_correctness(prompts, completions, answer, **kwargs):
    """
    Returns:
    - rewards: list of floats representing reward per sample
    """
    # Regex to extract predicted answer (assumes single digit 0-3 inside <answer> tags)
    match_format = re.compile(r"<answer>\s*([0-3])\s*</answer>", re.IGNORECASE)
    
    # Extract responses from completions
    responses = [comp[0]["content"] for comp in completions]
    
    # Extract predicted answers using regex
    extracted_responses = [
        guess.group(1) if (guess := match_format.search(r)) is not None else None
        for r in responses
    ]
    
    # Handle answer parameter (assuming it's a list of correct answers)
    extracted_answers = [str(a) for a in answer]
    
    rewards = []
    # Compute rewards
    for pred, true_answer in zip(extracted_responses, extracted_answers):
        if pred is None:
            score = 0.0  # No prediction found
        elif pred == true_answer:
            score = 1.0   # Exact match
        else:
            score = 0.0  # Changed from -0.5 to 0.0 for incorrect answer
        rewards.append(score)
    
    return rewards

In [42]:
# 2. Formating reward
def reward_format(completions, **kwargs):
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<answer>.*?</answer><explanation>.*?</explanation>"  # Remove ^ and $
    completion_contents = [completion[0]["content"] for completion in completions]
    matches = [re.search(pattern, content, re.DOTALL) for content in completion_contents]  # Use re.search instead of re.match
    return [1.0 if match else 0.0 for match in matches]


In [43]:
# 3. Explanation length reward (encourages detailed explanations)
def reward_explanation_length(completions, **kwargs):
    pattern = r"<explanation>(.*?)</explanation>"
    completion_contents = [completion[0]["content"] for completion in completions]
    rewards = []
    for content in completion_contents:
        match = re.search(pattern, content, re.DOTALL)
        if match:
            explanation = match.group(1).strip()
            # Reward based on explanation length (20-200 chars is good)
            length = len(explanation)
            if 20 <= length <= 200:
                rewards.append(1.0)
            elif length > 10:
                rewards.append(0.5)  # Partial credit for short explanations
            else:
                rewards.append(0.0)
        else:
            rewards.append(0.0)
    return rewards

In [44]:
from trl import GRPOTrainer, GRPOConfig
from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
     min_p = 0.1,
     top_p = 1.0,
     top_k = -1,
     seed = 3407,
     stop = [tokenizer.eos_token],
     include_stop_str_in_output = True,
)

    #per_device_train_batch_size=2,     # A safe value for 16GB VRAM with Unsloth
    #gradient_accumulation_steps=4, 
    #bf16=True,                         # Unsloth uses bfloat16 by default, but good to be explicit.

training_args = GRPOConfig(
    output_dir="./grpo-medmcqa-qwen-4b",
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = 1024,
    max_completion_length = 256,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 400,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
)

In [45]:
trainer = GRPOTrainer(
    model=model,
    processing_class = tokenizer,
    reward_funcs=[reward_correctness,reward_format,reward_explanation_length],
    args=training_args,
    train_dataset=train,
    eval_dataset=validation,
)

In [46]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 40,000 | Num Epochs = 1 | Total steps = 400
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 34,865,152 of 1,755,440,128 (1.99% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / reward_correctness / mean,rewards / reward_correctness / std,rewards / reward_format / mean,rewards / reward_format / std,rewards / reward_explanation_length / mean,rewards / reward_explanation_length / std
1,0.001300,0.625000,0.176777,94.625000,15.000000,197.000000,0.000000,94.625000,15.000000,197.000000,0.333438,0.000000,0.000000,0.000000,0.000000,0.625000,0.231455
2,0.001500,0.687500,0.088388,66.875000,11.000000,189.000000,0.000000,66.875000,11.000000,189.000000,0.370866,0.000000,0.000000,0.000000,0.000000,0.687500,0.372012
3,0.002000,0.375000,0.353553,66.875000,11.000000,256.000000,0.125000,39.857143,11.000000,101.000000,0.506926,0.000000,0.000000,0.000000,0.000000,0.375000,0.443203
4,0.001800,0.312500,0.441942,57.500000,11.000000,256.000000,0.125000,29.142859,11.000000,84.000000,0.458203,0.000000,0.000000,0.000000,0.000000,0.312500,0.458063
5,0.001100,0.562500,0.265165,75.500000,11.000000,185.000000,0.000000,75.500000,11.000000,185.000000,0.270065,0.000000,0.000000,0.000000,0.000000,0.562500,0.320435
6,0.002000,0.437500,0.088388,103.375000,11.000000,249.000000,0.000000,103.375000,11.000000,249.000000,0.510750,0.000000,0.000000,0.000000,0.000000,0.437500,0.176777
7,0.001700,0.437500,0.441942,65.000000,11.000000,149.000000,0.000000,65.000000,11.000000,149.000000,0.435375,0.000000,0.000000,0.000000,0.000000,0.437500,0.417261
8,0.002500,0.437500,0.265165,44.250000,11.000000,113.000000,0.000000,44.250000,11.000000,113.000000,0.621653,0.000000,0.000000,0.000000,0.000000,0.437500,0.417261
9,0.001900,0.562500,0.441942,136.500000,11.000000,256.000000,0.125000,119.428574,11.000000,250.000000,0.479365,0.000000,0.000000,0.000000,0.000000,0.562500,0.417261
10,0.001900,0.437500,0.265165,84.875000,11.000000,256.000000,0.125000,60.428574,11.000000,108.000000,0.473609,0.000000,0.000000,0.000000,0.000000,0.437500,0.417261


TrainOutput(global_step=400, training_loss=0.002241788605169859, metrics={'train_runtime': 2781.0966, 'train_samples_per_second': 1.151, 'train_steps_per_second': 0.144, 'total_flos': 0.0, 'train_loss': 0.002241788605169859})

In [52]:
text = "The most common organism causing urinary tract infections is:\nOptions:\n0. Staphylococcus aureus\n1. Escherichia coli\n2. Pseudomonas aeruginosa\n3. Enterococcus faecalis"

In [53]:
# Answer without GRPO

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.1,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

'\nThe most common organism causing urinary tract infections (UTIs) is:\n\n**1. Escherichia coli (E. coli)**\n\nE. coli is the leading cause of UTIs, particularly in women, due to its ability to adhere to the urinary tract lining and produce toxins that damage the tissue.'

In [54]:
model.save_lora("grpo_saved_lora")

In [55]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

In [56]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": text},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.1,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"1</answer><explanation>Ans. is 'b' i.e., Escherichia coli</explanation>"

In [57]:
!tar -czvf GRPO_Model.tar.gz ./grpo-medmcqa-qwen-4b

./grpo-medmcqa-qwen-4b/
./grpo-medmcqa-qwen-4b/checkpoint-100/
./grpo-medmcqa-qwen-4b/checkpoint-100/special_tokens_map.json
./grpo-medmcqa-qwen-4b/checkpoint-100/tokenizer.json
./grpo-medmcqa-qwen-4b/checkpoint-100/added_tokens.json
./grpo-medmcqa-qwen-4b/checkpoint-100/adapter_config.json
./grpo-medmcqa-qwen-4b/checkpoint-100/optimizer.pt
./grpo-medmcqa-qwen-4b/checkpoint-100/merges.txt
./grpo-medmcqa-qwen-4b/checkpoint-100/README.md
./grpo-medmcqa-qwen-4b/checkpoint-100/chat_template.jinja
./grpo-medmcqa-qwen-4b/checkpoint-100/adapter_model.safetensors
./grpo-medmcqa-qwen-4b/checkpoint-100/scaler.pt
./grpo-medmcqa-qwen-4b/checkpoint-100/tokenizer_config.json
./grpo-medmcqa-qwen-4b/checkpoint-100/training_args.bin
./grpo-medmcqa-qwen-4b/checkpoint-100/vocab.json
./grpo-medmcqa-qwen-4b/checkpoint-100/trainer_state.json
./grpo-medmcqa-qwen-4b/checkpoint-100/rng_state.pth
./grpo-medmcqa-qwen-4b/checkpoint-100/scheduler.pt
./grpo-medmcqa-qwen-4b/README.md
./grpo-medmcqa-qwen-4b/checkpoin